# 3.1. Serving an encrypted model
**protecting privacy and IP simultaneously**


### Use case: Skin cancer detection

Many people regularly check their skin for changes. Not all moles, patches, and rashes are cancerous. In this tutorial we'll train a model to differenciate between benign keratosis and melanoma (type of skin cancer).

Seborrheic keratosis is a noncancerous condition that can look a lot like melanoma. About 83 million Americans have seborrheic keratosis. About 5 percent of all new cancer cases in the United States are melanoma, a potentially deadly form of skin cancer. With prompt treatment, more than 91 percent of melanoma patients will survive 5 years or more after their first diagnosis. (Source: https://www.medicalnewstoday.com/articles/320742.php)


## Train a model

The first thing we need is a model, we can train a model using **regular pytorch**! If you're familiar with pytorch feel free to skip this part of the tutorial.

Most of the code used in this part of the tutorial is actually at `skin_cancer_model_utils.py` have a look at it if you want to know in more details what's going on in each subsection of this notebook.

### Dataset

We'll use this [kaggle dataset](https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000/). You need to download this dataset before running this example.

We use the code available at [this kaggle kernel](https://www.kaggle.com/kmader/dermatology-mnist-loading-and-processing) by Kevin Mader and in [this tutorial](https://towardsdatascience.com/skin-cancer-classification-with-machine-learning-c9d3445b2163) by Nyla Pirani to preprocess the data.

In the previous tutorial we served a CNN for classifying images with different 2 types of skin deseases: benign keratosis and melanoma (type of skin cancer). In this tutorial we show how to serve this model on a **encrypted way** on Grid.

### Dependencies for training the model

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torchvision.transforms as transforms

import skin_cancer_model_utils as scmu

# Path where the data is stored. Change this if needed!
scmu.DATASET_PATH

**Read dataset**

In [ ]:
df = scmu.read_skin_cancer_dataset()
df['cell_type'].value_counts()

**Visualize dataset**

In [ ]:
# Get samples from each class
samples = df.groupby('cell_type').apply(lambda x: x.sample(3))

# Plot samples
fig = plt.figure(figsize=(12, 8))
columns = 3
rows = 2
for i in range(len(samples)):
    image = mpimg.imread(samples["path"].iloc[i])
    fig.add_subplot(rows, columns, i + 1)
    plt.imshow(image)
    title = "{} ({})".format(samples["cell_type_idx"].iloc[i], samples["cell_type"].iloc[i])
    plt.title(title)
plt.show()

**Prepare data for training**

In [ ]:
train_df, valid_df, test_df = scmu.split_data(df)

In [ ]:
# Parameters for the dataloader
input_size = 32
params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 6}

# Calculate train_mean and train_std
training_set = scmu.Dataset(train_df, transform=transforms.Compose([
                               transforms.Resize((input_size, input_size)),
                               transforms.ToTensor()]))
training_generator = torch.utils.data.DataLoader(training_set, **params)
train_mean, train_std = scmu.calculate_mean_and_std(training_generator)

In [ ]:
train_mean, train_std

In [ ]:
# Create dataloaders
training_set = scmu.Dataset(train_df, transform=scmu.transform(input_size, train_mean, train_std))
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = scmu.Dataset(valid_df, transform=scmu.transform(input_size, train_mean, train_std))
validation_generator = torch.utils.data.DataLoader(validation_set, **params)

test_set = scmu.Dataset(test_df, transform=scmu.transform(input_size, train_mean, train_std))
test_generator = torch.utils.data.DataLoader(test_set, **params)

**Implement model**

In [ ]:
model = scmu.make_model()
model.eval()

**Train model**

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

In [ ]:
train_metrics, valid_metrics = scmu.train(model, epochs=10, optimizer=optimizer,
                                          training_generator=training_generator,
                                          validation_generator=validation_generator)

In [ ]:
train_metrics

In [ ]:
(train_errors, train_accs), (valid_errors, valid_accs) = (train_metrics, valid_metrics)

In [ ]:
plt.plot([train_error for train_error in train_errors], label ='Training error')
plt.plot([valid_error for valid_error in valid_errors], label ='Validation error')
plt.legend()
plt.show()
plt.plot([train_acc for train_acc in train_accs], label ='Training accuracy')
plt.plot([valid_acc for valid_acc in valid_accs], label ='Validation accuracy')
plt.legend()
plt.show()

**Evaluate model**

In [ ]:
scmu.test(model, test_generator)

In [ ]:
torch.save(model.state_dict(), "skin-cancer-detection-model")

## Serve model

In [ ]:
# Import dependencies
import grid as gr
from grid import syft as sy
import torch as th
import skin_cancer_model_utils as scmu

# Hook torch
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
grid_server = gr.WebsocketGridClient(hook, "http://localhost:3001", id="alice")
patient_server = gr.WebsocketGridClient(hook, "http://localhost:3000", id="bob")
hospital_server = gr.WebsocketGridClient(hook, "http://localhost:3002", id="james")
crypto_provider = gr.WebsocketGridClient(hook, "http://localhost:3003", id="dan")

grid_server.connect()
patient_server.connect()
hospital_server.connect()
crypto_provider.connect()

# Connect nodes to each other
gr.connect_all_nodes([grid_server, patient_server, hospital_server, crypto_provider])

In [ ]:
df = scmu.read_skin_cancer_dataset()
train_df, valid_df, test_df = scmu.split_data(df)

params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 6}

# These values are from 3.1.
input_size = 32
train_mean = train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

test_set = scmu.Dataset(test_df, transform=scmu.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, **params)

data, target = next(iter(test_generator))

### Making a model ready to be served and encrypted

In order to serve the model it needs to be serializable. A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. You can learn more about plans in [Syft's tutorials](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2008%20-%20Introduction%20to%20Plans.ipynb).


### Define Model

Let's load the model we just trained.

In [ ]:
model = scmu.make_model(is_plan=True)
model.load_state_dict(th.load("skin-cancer-detection-model"))

In [ ]:
model.build(data)

In [ ]:
scmu.test(model, test_generator)

## Serve model

In [ ]:
model.encrypt(patient_server, hospital_server, crypto_provider=crypto_provider)

In [ ]:
grid_server.serve_encrypted_model(model)

In [ ]:
x_sh = data.encrypt(patient_server, hospital_server, crypto_provider=crypto_provider)

### Get a copy of the private model

In [ ]:
# Fetch plan
fetched_plan = me.fetch_plan("convnet", alice, copy=True)

### Run encrypted inference

In [ ]:
%%time
print(fetched_plan(x_sh).get().float_prec())

In [ ]:
model = Net()

model.build(data)

In [ ]:
# encrypt the model

# share -> encrypt
model.encrypt(*workers, crypto_provider=crypto_provider)

In [ ]:
bob.serve_model("skin-cancer", model, allow_download=True, allow_remote_inference=False)

In [ ]:
data.encrypt(*workers, crypto_provider=crypto_provider)

In [ ]:
encrypted_model = bob.download_model("skin-cancer")

encrypted_model(data).request_decryption().float_prec()

In [ ]:
# TODOS
# ====================
# Internal changes: Syft interface -> Grid client
# fix_prec().share(*workers) == encrypt(*workers)
# get.float_prec() == request_decryption()
# allow_remote_inference -> allow_remote_inference
# allow_download -> allow_download
# bob.download -> bob.download_model()


# bob (public serving demo) -> app_company
# bob (private serving demo) -> ai_inc, cloud_server, ...

# encrypted demo:
# crypto_provider
# grid_server (hosts the model)
# patient_server (share holder 1)
# hospital_server (share holder 2)

model.encrypt(hospital_server, patient_server, crypto_provider=crypto_provider)
grid_server.serve_model("skin-cancer", model, allow_download=True, allow_remote_inference=False)

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

## Star PySyft on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

[Star PySyft](https://github.com/OpenMined/PySyft)

## Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

## Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

[PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
[Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

## Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)